# 공부하면서 배우는 EDA (1)

**작성자 DACON ID : TDA**

이 포스팅은 데이터분석, 통계학을 실제 데이터에 응용하면서 공부하기 위해 작성된 자료입니다.

이 포스팅은 가능하다면 하루에 한 편씩 제가 수행하는 만큼 시리즈로 제작할 예정입니다.

작성자는 데이터분석, 통계학에 능통하지 않습니다. 

오히려 초보에 가깝기 때문에 수준높은 분석은 아직 어렵습니다.

만약 이 포스팅을 읽으시는 분도 저와 유사한 수준의 지식과 기술을 가지고 계시다면, 이 포스팅을 읽으시면서 공부했던 것을 복습하는 시간이 될 것이라 생각합니다.

또한 얼마든지 잘못된 분석이 있을 수 있습니다.

이에 대해 코멘트를 남겨주시면 너무 감사하겠습니다.

>본 EDA는 DACONIO님의 baseline [EDA](https://dacon.io/competitions/official/235901/codeshare/5061?page=1&dtype=recent)와 데이터 분석 교과서 'Practical Statistics for Data Scientists, 2nd edition (Peter Bruce, Andrew Bruce, Peter Gedeck)'를 참고했습니다.

# 0. 라이브러리 임포트 및 데이터 불러오기

In [92]:
import pandas as pd 
import numpy as np 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
%matplotlib inline  
%precision 3


# 시각화 설정
colors = [plt.cm.Dark2(i) for i in range(20)]
mpl.rcParams.update({'font.size':18})

In [27]:

# csv 형식으로 된 데이터 파일을 읽어옵니다. 
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

train.head()

,id,title,odometer,location,isimported,engine,transmission,fuel,paint,year,target
0,0,Toyota RAV 4,18277,Lagos,Foreign Used,4-cylinder(I4),automatic,petrol,Red,2016,13665000
1,1,Toyota Land Cruiser,10,Lagos,New,4-cylinder(I4),automatic,petrol,Black,2019,33015000
2,2,Land Rover Range Rover Evoque,83091,Lagos,Foreign Used,6-cylinder(V6),automatic,petrol,Red,2012,9915000
3,3,Lexus ES 350,91524,Lagos,Foreign Used,4-cylinder(I4),automatic,petrol,Gray,2007,3815000
4,4,Toyota Venza,94177,Lagos,Foreign Used,6-cylinder(V6),automatic,petrol,Red,2010,7385000


# 1. 타겟 변수 분석

In [28]:
# 통계량을 먼저 살펴봅니다.
target = train['target']
target.describe()

count    1.015000e+03
mean     8.243204e+06
std      1.239567e+07
min      4.000000e+05
25%      2.535000e+06
50%      4.215000e+06
75%      8.927500e+06
max      1.500150e+08
Name: target, dtype: float64

## 1.1 위치추정
데이터가 주어졌을 때, 데이터를 살펴보는 가장 기초적인 단계는 *대푯값*을 구하는 것입니다.
이는 대부분의 값이 어디쯤에 위치하는지를 나타내는 값입니다.

먼저 **평균값**과 **중간값**을 구해봅시다.

In [38]:
# 평균과 중간값
print('평균값 : ', target.mean())
print('중간값 : ', target.median())
print('중간값과 평균값의 차이 : ', target.median() - target.mean())

평균값 :  8243204.450246305
중간값 :  4215000.0
중간값과 평균값의 차이 :  -4028204.450246305


데이터의 평균값과 중간값의 차이가 매우 큽니다.
그리고 그 차이 (중간값 - 평균값)이 음수로, 이는 평균값이 중간값보다 크다는 것을 의미합니다.

중간값은 이상치에 덜 민감한 반면, 평균값은 이상치에 민감합니다. 
따라서 두 값의 차이가 크다는 것은 **데이터에 이상치가 많을 수 있음**을 의미합니다.

이 상황에서는 평균값을 데이터의 대푯값으로 사용하기에 적절하지 않습니다.

이 경우 평균을 조금 변형한 **절사평균** 개념을 이용할 수 있습니다.
절사평균은 데이터들을 크기 순으로 나열한 후, 양끝에서 일정 비율만큼의 데이터를 삭제한 뒤 남은 값들로 평균을 구합니다.

아래의 코드는 각각 데이터의 양 끝에서 10%, 15%, 20%, 25% 제거하여 절사평균을 구하는 코드입니다.

절사평균을 구하기 위해서 `scipy.stats`의 `trim_mean` 함수를 이용합니다.

`trim_mean` 함수는 `trim_mean(데이터, 절사할 비율)` 형태로 이용하면 됩니다.

In [45]:
from scipy.stats import trim_mean

for trim in [0.0, 0.1, 0.15, 0.2, 0.25]:
    print(f'{trim*100}% 절사평균 : ',trim_mean(target, trim))

0.0% 절사평균 :  8243204.450246305
10.0% 절사평균 :  5653857.329643296
15.0% 절사평균 :  5252582.29113924
20.0% 절사평균 :  4967098.536945812
25.0% 절사평균 :  4764495.106090373


데이터를 5%씩 제거해갈수록 절사평균이 중간값에 가까워 집니다. 

특히 0% 절사평균(=평균)과 10% 절사평균 차이가 굉장히 큰 것으로 보아, 데이터 분포의 **양 옆 10%에 이상치가 위치할 가능성이 높습니다**.

## 1.2 변이추정
데이터의 특징을 요약하는 요소는 위치 외에도 **변이**가 있습니다.
변이는 데이터 값이 얼마나 밀집해 있는지 혹은 퍼져 있는지를 나타내는 **산포도**를 나타냅니다.

변이추정의 기본은 **편차**를 기본으로 합니다.
우리에게 익숙한 **분산** 또는 **표준편차**가 주로 이용됩니다.

In [48]:
print('분산 : ', target.var())
print('표준편차 : ', target.std())

분산 :  153652724096224.1
표준편차 :  12395673.603972642


표준편차가 굉장히 큽니다.
그러나 표준편차의 정의에 평균값이 포함된다는 점을 기억합시다.

앞서 평균값이 데이터의 대푯값으로 적절하지 않을 수 있다는 점을 상기해볼 때, 표준편차 또한 데이터의 변이를 나타내기에 적절하지 않을 수 있습니다.

이때 사용할 수 있는 값이 중간값의 **중위절대편차(MAD)** 입니다.
중위절대편차는 중위값과의 편차에 대한 중위값입니다.
중위절대편차의 수식적 정의는 다음과 같습니다.

$$MAD = median(|x_1 - m|, |x_2 - m|, \ldots, |x_N - m|)$$
여기서 $m$은 데이터의 중위값 입니다.
중위절대편차는 중위값과 마찬가지로 이상치에 대해 덜 민감합니다.

아래의 코드는 데이터의 중위절대편차를 구하는 코드입니다.
중위절대편차를 계산하기 위해 `statsmodels.api`의    `robust.scale.mad` 함수를 이용합니다.

In [70]:
# 중위표준편차 (MAD)는 statsmodels.api 패키지에 있는 함수로 구할 수 있다.
import statsmodels.api as sm

print('중위절대편차 : ',sm.robust.scale.mad(target))

중위절대편차 :  3409985.1025628843


In [64]:
# 구간 (m-mad, m+mad)을 구해 중위값 근방의 주요 데이터들이 속한 구간을 구해보자.
target.median() - sm.robust.scale.mad(target), target.median() + sm.robust.scale.mad(target)

(805014.897, 7624985.103)

MAD 값이 표준편차에 비해 훨씬 작은 값을 가지는 것을 알 수 있습니다.

중간값(4,215,000)을 기준으로 중위절대편차 한 단위씩 좌우로 위치한 구간을 구해보면 (805,015, 7,624,985)이 나옵니다.

어쩌면 이 구간 내의 가격에 속한 데이터들이 그나마 적정 가격선인 데이터이고, 이 구간 외에 있는 데이터들은 지나치게 가격이 비싼 데이터일 가능성이 있습니다.

**추후에 입력변수들에 대한 전처리와 분석이 마친 뒤, 이들 구간 밖에 있는 데이터들을 살펴볼 필요가 있겠습니다.**

> ***주의!***
> 분산, 표준편차, 중위절대편차는 모두 데이터가 정규분포에서 왔다고 가정합니다. 우리의 데이터가 정규분포를 따르지 않는다면 이 추정치들에 대한 해석은 적절하지 않을 수  있습니다.

## 1.3 사분위수범위
편차를 기준으로한 산포도는 데이터가 얼마나 모여있는지를 나타내지만, 데이터가 얼마나 퍼져있는지를 살펴보고 변이를 추정할 수도 있습니다.

이때 가장 기본이 되는 측도는 최댓값과 최솟값의 차이인 범위입니다.
그러나 최댓값이나 최솟값, 혹은 이들 근방에 있는 양 끝 값들은 이상치일 가능성이 높기 때문에, 이들을 지우고서 범위를 알아볼 수 있습니다.

이때 주로 사용하는 것이 **사분위수범위**입니다.
이는 데이터를 크기별로 나열했을 때, 하위 25%와 상위 25% 데이터를 제거해 구한 범위입니다.

`pandas`의 `DataFrame`은 사분위수를 구할 수 있는 `quantile` 메서드를 제공합니다.

In [67]:
# 최대, 최소, 범위
print('최댓값 : ', target.max())
print('최솟값 : ', target.min())
print('범위 : ', target.max() - target.min())

최댓값 :  150015008
최솟값 :  400000
범위 :  149615008


In [66]:
# 사분위수범위
print('상위 25% : ', target.quantile(0.75))
print('하위 25% : ', target.quantile(0.25))
print('사분위수 범위 : ',target.quantile(0.75) - target.quantile(0.25))

상위 25% :  8927500.0
하위 25% :  2535000.0
사분위수 범위 :  6392500.0


범위와 사분위수 범위 사이의 차이가 큽니다.
조금 더 자세히 살펴보면, 최댓값과 상위 25%값 사이의 차이가 굉장히 큽니다.
그리고 최솟값과 하위 25%값 사이의 차이는 상대적으로 작습니다.
어쩌면 **분포가 아래의 그림처럼 생겼을지도 모르겠습니다**.

<img src = 'figure1.png'>

이 직관에 의하면, 수집된 타겟 데이터는 정규분포를 따르지 않고있을 가능성이 큽니다.

# 1.4 왜도와 첨도
앞선 분석에서 우리 데이터가 정규분포에서 굉장히 벗어나있을 가능성이 높음을 수 있다는 직관을 얻었습니다.

왜도와 척도는 데이터의 분포가 정규분포에서 얼마나 벗어난 모양인지를 측정해주는 지표들입니다.

각각의 개념을 대략적으로 살펴보고 그 값을 `Python`으로 구해 분석해보겠습니다.

> **왜도(Skewness)**
> 
>왜도는 정규분포와 비교해서 확률변수의 분포가 얼마나 치우쳐 있는지의 정도를 나타냅니다.
>- 왜도가 <span style="color:green">**0**</span>인 경우 확률변수가 <span style="color:green">**정규분포**</span>를 에 가깝습니다.
>- 왜도가 <span style="color:red">**양수**</span>인 경우 확률분포가 <span style="color:red">**오른쪽**</span>으로 치우칩니다.
>- 왜도가 <span style="color:blue">**음수**</span>인 경우 확률분포가 <span style="color:blue">**왼쪽**</span>으로 치우칩니다.

![Skewness](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Negative_and_positive_skew_diagrams_%28English%29.svg/700px-Negative_and_positive_skew_diagrams_%28English%29.svg.png)
사진출처 : [위키백과 왜도(비대칭도)](https://ko.wikipedia.org/wiki/%EB%B9%84%EB%8C%80%EC%B9%AD%EB%8F%84)

In [68]:
# Dataframe의 skew() 메서드를 사용하면 왜도를 알 수 있다.
print('왜도 : ',target.skew())

왜도 :  4.9552768084421


왜도 값이 거의 5에 가깝습니다.
따라서 데이터는 **오른쪽으로 매우 치우쳐져** 있습니다.

해석을 하자면 가격이 저렴한 매물은 많지만, 가격이 비싼 매물은 희소하다는 의미가 됩니다.

> **첨도(Kurtosis)**
> 
>첨도는 확률분포의 꼬리가 얼마나 두꺼운지를 나타냅니다. 
>- 첨도가 <span style="color:green">**3**</span>인 경우 확률변수가 <span style="color:green">**정규분포**</span>에 가깝습니다.
>- 첨도가 <span style="color:red">**3보다 큰**</span> 경우 확률분포가 정규분포보다 <span style="color:red">**꼬리가 두껍습니다.**</span>
>- 첨도가 <span style="color:blue">**3보다 작은**</span>인 경우 확률분포가 정규분포보다 <span style="color:blue">**꼬리가 얇습니다.**</span>

![Kurtosis](https://upload.wikimedia.org/wikipedia/commons/thumb/3/33/Standard_symmetric_pdfs.svg/440px-Standard_symmetric_pdfs.svg.png)
사진출처 : [위키백과 첨도](https://ko.wikipedia.org/wiki/%EC%B2%A8%EB%8F%84)

In [69]:
# Dataframe의 kurt() 메서드를 사용하면 왜도를 알 수 있다.
print('첨도 : ', target.kurt())

첨도 :  35.55992197898911


첨도의 값이 35에 가깝습니다. 따라서 데이터는 정규분포에 비해 꼬리가 매우 두껍습니다.

일반적으로 첨도가 크면 **데이터에 이상치가 많은 것으로 생각합니다**.

정리해봅시다.
- 왜도가 굉장히 크다. 데이터가 정규분포로부터 굉장히 벗어나 있다. 따라서 **정규분포를 가정하는 많은 통계기법들은 소용이 없을 수 있다**. 예를들어, 이 데이터는 평균값을 데이터의 대푯값으로 사용하기에 부적합하다.
- 첨도가 굉장히 크다. 데이터에 이상치가 많을 수 있다. 이 경우 **이상치 탐치 알고리즘**을 적용해 데이터 전처리를 시도해볼 수 있다.

## 1.5 분석자동화

지금까지 위치분석, 변이분석, 사분위수범위, 왜도와 첨도 등을 통해 타겟변수를 분석해보았습니다.

타겟변수는 수치형변수이므로, 지금까지의 분석 방법은 타겟변수 뿐만 아니라 입력변수에 속한 수치형변수들을 분석하는 데에 사용할 수 있을 것입니다.

따라서 위 모든 분석과정을 하나의 함수로 작성해 다음부터는 수치형변수 데이터를 입력하기만 하면 각각의 분석값을 출력해주도록 하면 편리할 것입니다.

여기에는 각자의 아이디어를 무궁무진하게 응용할 수 있을 것입니다. 

In [97]:
# 수치형변수 분석기
from scipy.stats import trim_mean
import statsmodels.api as sm



def numerical_analysis(dataframe):
    # 분석을 통해 얻어지는 결과들은 dictionary에 저장합니다.
    analysis = {}

    # 위치분석
    analysis['평균'] = dataframe.mean()
    analysis['중위값'] = dataframe.median()
    
    for trim in [0.0, 0.1, 0.15, 0.2, 0.25]:
        analysis[f'{trim*100}% 절사평균'] = trim_mean(dataframe, trim)
    
    # 변이분석
    analysis['분산'] = dataframe.var()
    analysis['표준편차'] = dataframe.std()
    analysis['중위절대편차'] = sm.robust.scale.mad(dataframe)

    analysis['-1sigma'] = analysis['평균'] - analysis['표준편차']
    analysis['+1sigma'] = analysis['평균'] + analysis['표준편차']

    analysis['-1MAD'] = analysis['중위값'] - analysis['중위절대편차']
    analysis['+1MAD'] = analysis['중위값'] + analysis['중위절대편차']

    # 범위분석
    analysis['최댓값'] = dataframe.max()
    analysis['최솟값'] = dataframe.min()
    analysis['범위'] = analysis['최댓값'] - analysis['최솟값']
    
    analysis['75분위수'] = dataframe.quantile(0.25) 
    analysis['25분위수'] = dataframe.quantile(0.75)
    analysis['사분위수범위'] = analysis['75분위수'] - analysis['25분위수']

    # 왜도와 첨도
    analysis['왜도'] = dataframe.skew()
    analysis['첨도'] = dataframe.kurt()

    return analysis

In [98]:
numerical_analysis(target)

{'평균': 8243204.450,
 '중위값': 4215000.000,
 '0.0% 절사평균': 8243204.450,
 '10.0% 절사평균': 5653857.330,
 '15.0% 절사평균': 5252582.291,
 '20.0% 절사평균': 4967098.537,
 '25.0% 절사평균': 4764495.106,
 '분산': 153652724096224.094,
 '표준편차': 12395673.604,
 '중위절대편차': 3409985.103,
 '-1sigma': -4152469.154,
 '+1sigma': 20638878.054,
 '-1MAD': 805014.897,
 '+1MAD': 7624985.103,
 '최댓값': 150015008,
 '최솟값': 400000,
 '범위': 149615008,
 '75분위수': 2535000.000,
 '25분위수': 8927500.000,
 '사분위수범위': -6392500.000,
 '왜도': 4.955,
 '첨도': 35.560}

## 1.6 정리

지금까지 타겟 데이터의 대푯값을 추정하는 방법을 통해 데이터를 요약하고 분석해보았습니다.

분석 내용을 정리해봅시다.

> **위치분석결과** 
> 
> - 평균이 중위값보다 매우 크다. 데이터 중에 비정상적으로 큰 값이 존재할 수 있다.
> - 평균과 10%절사평균의 차이가 굉장히 크다. 양 끝 10% 데이터에 이상치가 존재할 가능성이 높다.
> 
> **변이분석결과**
> 
> - 분산과 표준편차가 비이상적으로 크다. 평균이 대푯값으로서 적절하지 않을 수 있다.
> - 중위값이 대푯값으로 적절하다고 생각하고 1MAD 단위 구간을 구하면, 약 (800000 , 7600000)이다.
> 
> **범위분석결과**
> 
> - 범위와 사분위수범위 간의 차가 굉장히 크다. 데이터에 이상치가 존재할 가능성이 높다.
> - 최댓값과 75분위수의 차이가 굉장히 크고 최솟값과 25분위수 사이의 차이는 상대적으로 작다. 이는 데이터의 분포가 작은값 쪽으로 많이 쏠린 형태임을 시사한다.
> 
> **왜도첨도분석**
> 
> - 왜도가 약 5(>0)이다. 데이터가 오른쪽으로 굉장히 치우쳐 있다.
> - 첨도가 약 35(>3)이다. 데이터의 꼬리가 굉장히 두꺼워, 이상치가 많을 가능성이 높다. 

분석 결과를 총합해 요약하자면, **타겟 데이터는 정규분포에서 많이 벗어나 있고, 이상치가 존재한다는 신호가 많다**고 할 수 있겠습니다.

이를 통해 **타겟 데이터가 어떤 분포를 띄는게 적절할지 생각**해볼 수 있겠습니다.

아무래도 타겟 데이터가 정규분포를 띄어야 하는게 타당하다고 생각한다면, 샘플을 더 수집할 수는 없으니 데이터 전처리를 설계할 때 타겟 데이터에 대해 이상치탐지 알고리즘을 사용할 것을 재고해볼 수 있겠습니다.

혹은 타겟 데이터가 정규분포가 아닌 다른 분포를 띄어야 할 수도 있겠다 추론할 수 있습니다.
이를 분석하기 위해서는 더욱 깊은 통계지식과 직관, 그리고 타겟 데이터에 대한 도메인지식이 필요할 수 있습니다.

본 포스팅에서 사용된 방법들은 다른 수치형 변수에 대해서도 동일하게 응용될 수 있습니다.

또한 추후에 데이터전처리 과정을 거친 다음 다시 동일한 분석방법을 적용해볼 수 있습니다.

그때 데이터전처리 전후의 분석 변화를 살펴보는 것도 유의미한 지식이 될 것입니다.

다음 번에는 타겟 데이터가 전반적으로 어떻게 분포하고 있는지 다양한 시각화 도구들을 통해 분석해보도록 하겠습니다.

지금까지 읽어주셔서 감사합니다 :)